# Imports

In [ ]:
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from sklearn.model_selection import train_test_split
import unicodedata
from transformers import BertTokenizerFast, BertModel
import re
from collections import defaultdict, Counter
import random
from sklearn.metrics import classification_report
import pandas as pd

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [24]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [25]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [26]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [27]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [28]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['Pasa que yo tengo 30 años.', '¿Quién?', 'Y ahora va a empezar a largar una tras otra indirecta para que vayas a comprar un pollo al spiedo.', 'Hablé más bajo.', 'Ah, está impresionadísimo...hace tanto calor ahí adentro.', 'Ay, son...', 'Felipe: habla Sergio.', 'Ella es de una naturaleza muy peculiar, muy nerviosa. Pero vos, ¿estás segura de lo que estás diciendo? Porque yo no puedo creerlo, te juro, no puedo creerlo.', 'Aquí pasaron cosas mucho más importantes hoy.', 'Nosotros tenemos siempre la casa oscura. Mamá Cora se pasa el día mirando la televisión.']
✅ Se extrajeron 1000 frases de Relatos Salvajes.


### Fuente 6 (beta): Mixture de oraciones

In [29]:
cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)

['¿Qué generó los píxeles brillantes durante las exposiciones? Uso Google Keep para tomar notas rápidas.',
 '¿En qué posición juega Gómez? ¿Cuáles son las características de la novela polifónica?',
 'Mi amigo de Panamá, llamado Luis, es un exitoso hombre de negocios. ¿Cuándo recuperó Wollstonecraft su prestigio?',
 '¿Quiénes eran los personajes que podían verse habitualmente en las obras shunga? Quiero una doble Whopper de Burger King.',
 '¿Quién está lesionado? ¿A qué institución internacional se ha adherido la república balcánica?']

## Juntamos las fuentes

In [30]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['Tampoco seguía con mayor interés la asignatura de artes plásticas en la escuela.',
 'Der Fotograf Guillermo Kahlo [El padre de Frida.',
 'En mayo de 2016, fue juzgado junto a su padre por tres cargos de evasión fiscal.',
 '¿Cuándo tendrán lugar las elecciones?',
 '¿Qué juego de mesa practica? Perderse en una tienda IKEA es muy fácil.']

## Importamos el modelo

In [31]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [32]:
procesar_oracion("¿y vos ¿como?", tokenizer)

[{'word': 'y',
  'token': 193,
  'prev_token': -1,
  'next_token': 63299,
  'has_accent': 0,
  'position': 0.0,
  'starting_punctuation_type': 4,
  'ending_punctuation_type': 0,
  'capitalization_type': 0},
 {'word': 'vos',
  'token': 63299,
  'prev_token': 193,
  'next_token': 10225,
  'has_accent': 0,
  'position': 0.5,
  'starting_punctuation_type': 0,
  'ending_punctuation_type': 0,
  'capitalization_type': 0},
 {'word': 'como',
  'token': 10225,
  'prev_token': 63299,
  'next_token': -1,
  'has_accent': 0,
  'position': 1.0,
  'starting_punctuation_type': 4,
  'ending_punctuation_type': 3,
  'capitalization_type': 0}]

In [33]:
dataset = []
count = 0
for sentence in oraciones_raw:
    if count % 1000 == 0:
      print(f"vamos {count}")
    dataset.append(procesar_oracion(sentence, tokenizer))
    count += 1

flattened_dataset = [item for sublist in dataset for item in sublist]

vamos 0
vamos 1000
vamos 2000
vamos 3000
vamos 4000
vamos 5000
vamos 6000
vamos 7000
vamos 8000
vamos 9000
vamos 10000
vamos 11000
vamos 12000
vamos 13000
vamos 14000
vamos 15000
vamos 16000
vamos 17000
vamos 18000
vamos 19000
vamos 20000


In [34]:
X = [[item['token'], item['prev_token'], item['next_token'],  item['has_accent'], item['position']] for item in flattened_dataset]
y_capitalization = [item['capitalization_type'] for item in flattened_dataset]
y_ending_punctuation = [item['ending_punctuation_type'] for item in flattened_dataset]
y_starting_punctuation = [item['starting_punctuation_type'] for item in flattened_dataset]

print("Distribución de clases en y_capitalization:")
print(Counter(y_capitalization))

print("Distribución de clases en y_ending_punctuation:")
print(Counter(y_ending_punctuation))

print("Distribución de clases en y_starting_punctuation:")
print(Counter(y_starting_punctuation))


Distribución de clases en y_capitalization:
Counter({0: 306455, 1: 96921, 3: 4417, 2: 1560})
Distribución de clases en y_ending_punctuation:
Counter({0: 374049, 2: 13627, 3: 11253, 1: 10424})
Distribución de clases en y_starting_punctuation:
Counter({0: 397536, 4: 11729, 2: 82, 3: 3, 1: 3})


In [35]:
undersample_freq = {
    0: 150000,
    1: 40000,
}
X_capitalization, y_capitalization = undersample(X, y_capitalization,
                                                  undersample_freq)

undersample_freq_punctuation = {
    0: 60000,
}
X_ending_punctuation, y_ending_punctuation = undersample(X, y_ending_punctuation,
                                            undersample_freq_punctuation)

undersample_freq_punctuation = {
    0: 200000,
}
X_starting_punctuation, y_starting_punctuation = undersample(X, y_starting_punctuation,
                                            undersample_freq_punctuation)

In [36]:
# ARBOL PARA CAPITALIZACION
from sklearn.ensemble import RandomForestClassifier

X_train_cap, X_test_cap, y_train_cap, y_test_cap = train_test_split(X_capitalization, y_capitalization, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=10)

clf.fit(list(X_train_cap), list(y_train_cap))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [37]:
# ARBOL PARA PUNTUACION INICIAL

X_train_punc_start, X_test_punc_start, y_train_punc_start, y_test_punc_start = train_test_split(X_starting_punctuation, y_starting_punctuation, test_size=0.2, random_state=42)
clf_punc_start = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=10)

clf_punc_start.fit(list(X_train_punc_start), list(y_train_punc_start))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [38]:
# ARBOL PARA PUNTUACION FINAL

X_train_punc_end, X_test_punc_end, y_train_punc_end, y_test_punc_end = train_test_split(X_ending_punctuation, y_ending_punctuation, test_size=0.2, random_state=42)
clf_punc_end = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=10)

clf_punc_end.fit(list(X_train_punc_end), list(y_train_punc_end))

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [39]:
score = clf.score(X_test_cap, y_test_cap)
print(f"Accuracy cap: {score}")

score = clf_punc_end.score(X_test_punc_end, y_test_punc_end)
print(f"Accuracy end: {score}")

score = clf_punc_start.score(X_test_punc_end, y_test_punc_end)
print(f"Accuracy start: {score}")

Accuracy cap: 0.8723594244310644
Accuracy end: 0.7826976548974346
Accuracy start: 0.6101463721735481


In [40]:

y_pred_cap = clf.predict(X_test_cap)
print("Classification report for capitalization:")
print(classification_report(y_test_cap, y_pred_cap))

y_pred_punc_end = clf_punc_end.predict(X_test_punc_end)
print("Classification report for ending punctuation:")
print(classification_report(y_test_punc_end, y_pred_punc_end))

y_pred_punc_start = clf_punc_start.predict(X_test_punc_start)
print("Classification report for starting punctuation:")
print(classification_report(y_test_punc_start, y_pred_punc_start))

Classification report for capitalization:
              precision    recall  f1-score   support

           0       0.88      0.98      0.92     30065
           1       0.85      0.53      0.65      7884
           2       0.89      0.44      0.59       324
           3       0.84      0.53      0.65       923

    accuracy                           0.87     39196
   macro avg       0.86      0.62      0.70     39196
weighted avg       0.87      0.87      0.86     39196

Classification report for ending punctuation:
              precision    recall  f1-score   support

           0       0.83      0.97      0.90     12017
           1       0.58      0.22      0.32      2082
           2       0.68      0.58      0.63      2739
           3       0.63      0.53      0.57      2223

    accuracy                           0.78     19061
   macro avg       0.68      0.58      0.60     19061
weighted avg       0.76      0.78      0.76     19061

Classification report for starting punctua

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and be

In [43]:
entrada = "cuál está la bolsa de valores"
df, salida = random_forest_predict_and_reconstruct(
        clf,
        clf_punc_start,
        clf_punc_end,
        entrada,
        tokenizer,
        verbose=False
)

print(df)
print(salida)

TypeError: 'RandomForestClassifier' object is not callable